## Create FHIRSTU3 SearchParameter Resource, CababilityStatement and Quick start text, and Searchparameter list. Using the Python FHIRClient Module and SearchParameter CSV file with source data.

### Prerequisites:

- Python 3.6 or greater


### Import FHIRClient and other libraries

In [9]:
%config IPCompleter.greedy=True

In [10]:
from fhirclient.models import searchparameter as SP
from fhirclient.models import capabilitystatement as CS
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.contactdetail as CD
from json import dumps
from requests import get, post, put
import os
from csv import reader as csvreader
from IPython.display import display, Markdown, HTML
from pprint import pprint
from collections import namedtuple
from pandas import read_csv
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape

#### Inspect ElementProperties as reference

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

In [23]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''
out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
#out_path = "C:/Users/Eric/Documents/Jan_2019_FHIR_Experience"
#sp_csv = 'SearchParams.csv'
sp_csv = 'Q_SearchParam.csv'
md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

fhir_term_server = 'http://test.fhir.org/r3'
fhir_test_server = 'http://test.fhir.org/r3'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "argo-q"
canon_base = "canon_base-for-argo-q/"
publisher = 'The Argonaut Project'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'https://github.com/argonautproject/questionnaire/issues'
                        )
none_list = ['', ' ', 'none', 'n/a', 'N/A']
sep_list = (',', ';', ' ', ', ', '; ')
search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]'
    )


### Get Search Parameter input data

In [26]:
data = read_csv(f'{in_path}{sp_csv}')
data

,Resource,Conformance Server,Parameter,Exists,Type,Expression,Modifiers,Comparators,Reference,Chains,Description,Example Query,foo
0,HealthcareService,SHALL,active,Y,token,HealthcareService.active,NaN,NaN,NaN,NaN,The Healthcare Service is currently marked as ...,GET [base]/HealthcareService?active=[code],NaN
1,Questionnaire,SHOULD,context-type-value,N,composite,On Questionnaire.useContext: context-type: cod...,NaN,NaN,NaN,NaN,�use context type and value assigned to the qu...,GET [base]/Questionnaire?context-type-value=[U...,NaN


In [27]:
with open(f'{in_path}{sp_csv}') as f:
    reader = csvreader(f)
    top_row = next(reader)
    top_row = [t.lower().split(' ') for t in top_row]
    top_row = ['_'.join(t) for t in top_row]

    Data = namedtuple("Data", top_row)
    data = [Data(*r) for r in reader]
    
for d in data:
    print(f'Resource = {d.resource}, Search Parameter = {d.parameter}')
    

Resource = HealthcareService, Search Parameter = active
Resource = Questionnaire, Search Parameter = context-type-value


### Create SP Resources by assigning CSV data to SP elements

In [28]:
sp_list = []



def split_string(s):
    for r in sep_list:
        s = s.replace(r, ' ')
    return s.split()

for d in data:
    #initiliaze
    sp = SP.SearchParameter()
    sp.date = D.FHIRDate(datetime.utcnow().isoformat())
    sp.publisher = publisher
    sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
    # print(d.parameter)
    sp.id = f'{base_id}-{d.resource.lower()}-{d.parameter}'
    sp.url = f'{canon_base}{sp.id}'
    sp.name = sp.id.replace('-','_')
    sp.derivedFrom = f'http://hl7.org/fhir/SearchParameter/{d.resource}-{d.parameter.lower()}' if d.exists == 'Y' else None
    # print(sp.derivedFrom)
    sp.status = 'active'
    sp.description = d.description
    sp.code = d.parameter.lower()
    sp.base = [d.resource]  # list
    sp.type = d.type.lower()
    # print(sp.type)
    sp.expression = d.expression
    sp.target = [] if d.reference in none_list else split_string(d.reference) # string as space separated list in csv
    # print(sp.target)
    # sp.multipleOr = True  add columns to csv for where needed or used otherwise leave blank
    # sp.multipleAnd = True  add columns to csv for where needed or used
    sp.comparator = [] if d.comparators in none_list else split_string(d.comparators) # string as space separated list in csv
    # print(sp.comparator)
    sp.modifier = [] if d.modifiers in none_list else split_string(d.modifiers) # string as space separated list in csv
    # print(sp.modifier)
    sp.chain = [] if d.chains in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.chain)
    # TODO sp.combo = [] if d.combos in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.combo)
    sp.conformance = d.conformance_server.upper()
    
    sp_list.append(sp)
    
    


### Display SP resources

In [30]:
for row,i in enumerate(sp_list):
    print(row,i.base)
    print(dumps(sp_list[1].as_json(), indent = 3))



0 ['HealthcareService']
{
   "id": "argo-q-questionnaire-context-type-value",
   "base": [
      "Questionnaire"
   ],
   "code": "context-type-value",
   "contact": [
      {
         "telecom": [
            {
               "system": "url",
               "value": "https://github.com/argonautproject/questionnaire/issues"
            }
         ]
      }
   ],
   "date": "2019-02-16T09:32:33.619803",
   "description": "\ufffduse context type and value assigned to the questionnaire",
   "expression": "On Questionnaire.useContext: context-type: code context: value.as(CodeableConcept)",
   "name": "argo_q_questionnaire_context_type_value",
   "publisher": "The Argonaut Project",
   "status": "active",
   "type": "composite",
   "url": "canon_base-for-argo-q/argo-q-questionnaire-context-type-value",
   "resourceType": "SearchParameter"
}
1 ['Questionnaire']
{
   "id": "argo-q-questionnaire-context-type-value",
   "base": [
      "Questionnaire"
   ],
   "code": "context-type-value",
   "

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles

In [31]:
r_type = "CareTeam"  # TODO  loop over all types in sp_list

template = env.get_template(md_template[0])
sp_bytype = [i for i in sp_list if i.base[0] == r_type]
display(Markdown(template.render(sp_bytype=sp_bytype,search_type=search_type)))
search_md = template.render(sp_bytype=sp_bytype,search_type=search_type)

with open(f'{out_path}{r_type}-search.md','w') as f:
    f.write(search_md)



### Quick Start
Below is an overview of the required set of RESTful FHIR interactions - for
example, search and read operations - for this profile. See the [Conformance requirements]
for a complete list of supported RESTful interactions for this IG.

#### Supported Searches

{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [32]:
# get types:
r_list = []
for i in sp_list:
    for j in i.base:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])
display(Markdown(template.render(sp_list=sp_list,r_list=r_list)))
searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)
with open(f'{out_path}searchparameters.md','w') as f:
    f.write(searchparameters_md)

---
title: Operations and Search Parameters
layout: default
topofpage: true
sectionnumbering: true
---

The following search parameters have been defined for the {{site.data.fhir.igName}} Implementation Guide.  For more information on the [FHIR RESTful search api]and the standard [Search Param Registry] see the FHIR specification.

<!-- Operations


  { % include list-simple-operationdefinitions.xhtml % }


-->

Search Parameter


**HealthcareService**
  - [active](SearchParameter-argo-q-healthcareservice-active.html)

**Questionnaire**
  - [context-type-value](SearchParameter-argo-q-questionnaire-context-type-value.html)

{% include link_list.md %}

### Update CapabilityStatement for Search
- instantiate new or existing CapStatement as fhir model
- add combination extensions
- add markdown descriptions from above

In [33]:
# instantiate new or existing CapStatement:

def interaction(code):
    return CS.CapabilityStatementRestResourceInteraction({'code' : code})

def search_cs(sp):
    template = env.get_template(md_template[2])
    return CS.CapabilityStatementRestResourceSearchParam(
    dict(
    name = sp.code,
    definition = sp.url,
    type = sp.type,
    documentation = template.render(sp=sp, search_type=search_type)
    )
    )

cs = CS.CapabilityStatement(dict(
                            date = now,
                            status = 'active',
                            acceptUnknown = 'both',
                            format = ['json','xml'],
                            fhirVersion = '3.0.1',
                            kind = 'requirements'     
                            ))



cs.rest = [CS.CapabilityStatementRest()]  
cs.rest[0].mode = 'server'
cs.rest[0].resource = []



print(r_list)    
for r in r_list:
    rest_resource = CS.CapabilityStatementRestResource()
    rest_resource.type = r
    rest_resource.interaction = [interaction(code) for code in ['read','vread', 'history-instance', 'search-type']]
    cs.rest[0].resource.append(rest_resource)
    rest_resource.searchParam = [search_cs(sp=sp) for sp in sp_list if sp.base[0] == r]
print(dumps(cs.as_json(),indent = 3))
    
    



NameError: name 'now' is not defined

### TODO

- Save SearchParameters and CapabilityStatements.
- Cycle through the quick starts
- Update the CSV files